In [1]:
!python3 cifar_train_lorot-E_BT.py --exp_str testingoutput --workers 8 --epochs 1 --gpu 0 --r_ratio 0.3 -a BarlowTwins 

usage: cifar_train_lorot-E_BT.py [-h] [--dataset DATASET] [-a ARCH]
                                 [--loss_type LOSS_TYPE] [--imb_type IMB_TYPE]
                                 [--imb_factor IMB_FACTOR]
                                 [--train_rule TRAIN_RULE]
                                 [--rand_number RAND_NUMBER]
                                 [--exp_str EXP_STR] [-j N] [--epochs N]
                                 [--start-epoch N] [-b N] [--lr LR]
                                 [--momentum M] [--wd W] [-p N]
                                 [--resume PATH] [-e] [--pretrained]
                                 [--seed SEED] [--gpu GPU]
                                 [--root_log ROOT_LOG]
                                 [--root_model ROOT_MODEL] [--r_ratio R_RATIO]
                                 [-x EXPERIMENT]
cifar_train_lorot-E_BT.py: error: unrecognized arguments: 0


In [1]:
import torchvision
import torch.nn as nn
import torch
import models
from torchsummary import summary

In [2]:
def off_diagonal(x):
    # return a flattened view of the off-diagonal elements of a square matrix
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

In [3]:
class BarlowTwins(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.backbone = torchvision.models.resnet50(zero_init_residual=True)
        self.backbone.fc = nn.Identity()

        # projector
        sizes = [2048] + [8192, 8192, 8192]
        layers = []
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=False))
            layers.append(nn.BatchNorm1d(sizes[i + 1]))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Linear(sizes[-2], sizes[-1], bias=False))
        self.projector = nn.Sequential(*layers)

        # normalization layer for the representations z1 and z2
        self.bn = nn.BatchNorm1d(sizes[-1], affine=False)

    def forward(self, y1, y2):
        z1 = self.projector(self.backbone(y1))
        z2 = self.projector(self.backbone(y2))

        # empirical cross-correlation matrix
        c = self.bn(z1).T @ self.bn(z2)

        return c

In [2]:
model = models.resnet32()

num_trans : 16


In [3]:
print(model)

ResNet_s(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affin